# Process Fitness Data
Notebook for the extraction and processing of fitness logs
### TOC:
* [Init Environment](#init_environment)
* [Extract Data](#extract_data)
* [Transform Data](#transform_data)
* [Load Data](#load_data)

***
## Init Environment <a class="anchor" id="init_environment"></a>

In [129]:
# imports
import pandas as pd
import datetime
import pprint

from pydrive.drive import GoogleDrive
from pydrive.auth import GoogleAuth
from pandleau import *

# misc objects
gauth = GoogleAuth()
drive = GoogleDrive(gauth)

# misc globals
path_to_conf = "./conf/"
path_to_logs = "./logs/"
path_to_tableau = "./tableau/"

file_tmp = "tmp_FY20 H1 Workout Tracker.xlsx"
file_out_weightlifting = "weightlifting_extract.hyper"
file_out_cardio = "cardio_extract.hyper"

lookup_weightlifting = {"Bench Press", "Deadlifts", "Shoulder Press", "Squat"}
columns_weightlifting = {
    'Rotation' : 'Rotation',
    '1RM' : 'Goal 1RM',
    'Workout' : 'Workout',
    'Exercise' : 'Exercise',
    'Week' : 'Week',
    'Sets' : 'Sets',
    'Reps' : 'Reps',
    '% 1RM (pace if running)' :  '% of Goal 1RM',
    'Actual Lift' :  'Actual Lift',
    'Date' : 'Date',
    'Time' : 'Time',
    'Bodyweight' : 'Bodyweight',
    'Notes' : 'Notes',
    'timestamp' : "Timestamp"
}
lookup_cardio = {"Run"}
columns_cardio = {
    'Rotation' : 'Rotation',
    'Workout' : 'Workout',
    'Exercise' : 'Exercise',
    'Week' : 'Week',
    '% 1RM (pace if running)' :  'Average Pace',
    'Actual Lift' :  'Total Distance',
    'Date' : 'Date',
    'Time' : 'Time',
    'Notes' : 'Notes',
    'timestamp' : "Timestamp"
}


print("SYSTEM INIT: {0}".format(
    datetime.datetime.strftime(datetime.datetime.now(), '%D %r')
))

SYSTEM INIT: 04/16/20 04:32:32 PM


In [130]:
# helper functions

def make_uid(row) :
    '''creates workout uid'''
    return str(int(row['Rotation'])) + "_" + str(int(row['Workout'])) + "." + str(int(row['Week']))

def make_datetime(r) :
    '''constructs datetime object for each workout'''
    if pd.notnull(r['Date']) :
        datetime_string = datetime.datetime.combine(r['Date'], r['Time'])
        return datetime_string

def make_1rm(x) :
    '''calculates a theoretical 1RM for each lift'''
    if pd.notnull(x) :
        r, w = x.split('x')[0], x.split('x')[1]
        return float(w)*(1 + int(r)/30)
    return None 
    
def make_conversion(df, out_file) :
    '''converts dataframe to tableau compliant format'''
    print(df.shape)
    tmp_df = pandleau(df)

    # remove if file exists, write out new file
    if os.path.isfile(os.path.join(path_to_tableau, out_file)) :
        os.remove(os.path.join(path_to_tableau, out_file))
    
    # tableau conversion
    tmp_df.to_tableau(
        os.path.join(path_to_tableau, out_file), 
        add_index=False
    )

    print("PYTHON: {0} conversion complete".format(os.path.join(path_to_tableau, out_file)))
    print("PYTHON: {0}".format(datetime.datetime.now().strftime("%d/%m/%Y %H:%M")))

def clean_logs() :
    '''cleans working directory of logs generated by conversion process'''
    # clean up log files
    log_path = os.path.join(os.getcwd(), "logs")

    log_files = [
        f for f in os.listdir("./")
        if os.path.isfile(os.path.join("./", f)) and ".log" in f
        or "hyper_db_" in f
    ]

    for file in log_files :
        os.rename(file, os.path.join(log_path, file))

    print("\nPYTHON: Directory cleaned")
    print("PYTHON: {0}".format(datetime.datetime.now().strftime("%d/%m/%Y %H:%M")))

***
## Extract Data <a class="anchor" id="extract_data"></a>

In [138]:
# extract raw log from google drive
mimetypes = {
    'application/vnd.google-apps.document': 'application/vnd.openxmlformats-officedocument.wordprocessingml.document', # Drive Document files as MS Word files.
    'application/vnd.google-apps.spreadsheet': 'application/vnd.openxmlformats-officedocument.spreadsheetml.sheet' # Drive Sheets files as MS Excel files.
}

file_list = drive.ListFile({'q': "'root' in parents and trashed=false"}).GetList();
for file1 in file_list:
    
    if file1['title'] == "FY20 H1 Workout Tracker" :
        
        download_mimetype = None
        if file1['mimeType'] in mimetypes:
            download_mimetype = mimetypes[file1['mimeType']]

        
        file1.GetContentFile("tmp_"+file1['title']+".xlsx", mimetype=download_mimetype) # write out to .xlsx as tmp file locally

In [132]:
# read in tmp file to dataframe

df_raw = pd.read_excel(file_tmp, skiprows=3)
print(df_raw.shape)
df_raw.head(3)

(207, 13)


,Rotation,1RM,Workout,Exercise,Week,Sets,Reps,% 1RM (pace if running),Actual Lift,Date,Time,Bodyweight,Notes
0,6.0,68,1.0,Shoulder Press,1.0,3.0,5,"65, 75, 85",5x57.5,2020-02-16,20:00:00,91.6,NaN
1,NaN,NaN,NaN,Shoulder Press,2.0,3.0,3,"70, 80, 90",4x60,2020-02-28,14:00:00,NaN,NaN
2,NaN,NaN,NaN,Shoulder Press,3.0,3.0,5 / 3 / 1,"75, 85, 95",1x65,2020-03-07,12:15:00,90.7,NaN


***
## Transform Data <a class="anchor" id="transform_data"></a>

* Create general calculated fields
    * Create workout uid
    * Format date and time
* Split out Weightlifting and Cardio
* Create exercise type specific calculated fields
    * format date and time columns

In [133]:
# drop junk data, retain only recorded exercises
df_raw = df_raw[(df_raw['Exercise'].isna()!=True) & (df_raw['Week'].isna()!=True)]

# propogate codes down for uid
df_raw[['Rotation', 'Workout']] = df_raw[['Rotation', 'Workout']].fillna(method='ffill')
df_raw['workout_uid'] = df_raw.apply(make_uid, axis=1)
df_raw.set_index('workout_uid', inplace=True)

# format date and time columns
df_raw['timestamp'] = df_raw.apply(make_datetime, axis=1)

df_raw.head()

,Rotation,1RM,Workout,Exercise,Week,Sets,Reps,% 1RM (pace if running),Actual Lift,Date,Time,Bodyweight,Notes,timestamp
workout_uid,,,,,,,,,,,,,,
6_1.1,6.0,68,1.0,Shoulder Press,1.0,3.0,5,"65, 75, 85",5x57.5,2020-02-16,20:00:00,91.6,NaN,2020-02-16 20:00:00
6_1.2,6.0,NaN,1.0,Shoulder Press,2.0,3.0,3,"70, 80, 90",4x60,2020-02-28,14:00:00,NaN,NaN,2020-02-28 14:00:00
6_1.3,6.0,NaN,1.0,Shoulder Press,3.0,3.0,5 / 3 / 1,"75, 85, 95",1x65,2020-03-07,12:15:00,90.7,NaN,2020-03-07 12:15:00
6_1.4,6.0,NaN,1.0,Shoulder Press,4.0,3.0,5,"40, 50, 60",2x60,2020-03-15,20:48:00,90.7,NaN,2020-03-15 20:48:00
6_2.1,6.0,148.5,2.0,Deadlifts,1.0,3.0,5,"65, 75, 85",4x120,2020-02-23,17:15:00,90.9,hangover,2020-02-23 17:15:00


In [134]:
# create seperate datasets

# weightlifting
map_weightlifting = df_raw['Exercise'].apply(lambda x: True if x in lookup_weightlifting else False)
df_weightlifting = df_raw[map_weightlifting==True][list(columns_weightlifting.keys())]
print(df_weightlifting.shape)

df_weightlifting.rename(columns = columns_weightlifting, inplace=True)
display(df_weightlifting.head(3))

# cardio
map_cardio = df_raw['Exercise'].apply(lambda x: True if x in lookup_cardio else False)
df_cardio = df_raw[map_cardio==True][list(columns_cardio.keys())]
print(df_cardio.shape)

df_cardio.rename(columns = columns_cardio, inplace=True)
display(df_cardio.head(3))

(100, 14)


,Rotation,Goal 1RM,Workout,Exercise,Week,Sets,Reps,% of Goal 1RM,Actual Lift,Date,Time,Bodyweight,Notes,Timestamp
workout_uid,,,,,,,,,,,,,,
6_1.1,6.0,68,1.0,Shoulder Press,1.0,3.0,5,"65, 75, 85",5x57.5,2020-02-16,20:00:00,91.6,NaN,2020-02-16 20:00:00
6_1.2,6.0,NaN,1.0,Shoulder Press,2.0,3.0,3,"70, 80, 90",4x60,2020-02-28,14:00:00,NaN,NaN,2020-02-28 14:00:00
6_1.3,6.0,NaN,1.0,Shoulder Press,3.0,3.0,5 / 3 / 1,"75, 85, 95",1x65,2020-03-07,12:15:00,90.7,NaN,2020-03-07 12:15:00


(29, 10)


,Rotation,Workout,Exercise,Week,Average Pace,Total Distance,Date,Time,Notes,Timestamp
workout_uid,,,,,,,,,,
6_5.1,6.0,5.0,Run,1.0,05:58:00,6.25km,2020-02-25,20:10:00,NaN,2020-02-25 20:10:00
6_5.2,6.0,5.0,Run,2.0,05:40:00,6.30km,2020-03-04,18:22:00,NaN,2020-03-04 18:22:00
6_5.3,6.0,5.0,Run,3.0,06:01:00,6.31km,2020-03-09,19:47:00,NaN,2020-03-09 19:47:00


In [135]:
# weightlifting specific processing
df_weightlifting['Calculated 1RM'] = df_weightlifting['Actual Lift'].apply(lambda x: make_1rm(x))
df_weightlifting.head(3)

,Rotation,Goal 1RM,Workout,Exercise,Week,Sets,Reps,% of Goal 1RM,Actual Lift,Date,Time,Bodyweight,Notes,Timestamp,Calculated 1RM
workout_uid,,,,,,,,,,,,,,,
6_1.1,6.0,68,1.0,Shoulder Press,1.0,3.0,5,"65, 75, 85",5x57.5,2020-02-16,20:00:00,91.6,NaN,2020-02-16 20:00:00,67.083333
6_1.2,6.0,NaN,1.0,Shoulder Press,2.0,3.0,3,"70, 80, 90",4x60,2020-02-28,14:00:00,NaN,NaN,2020-02-28 14:00:00,68.000000
6_1.3,6.0,NaN,1.0,Shoulder Press,3.0,3.0,5 / 3 / 1,"75, 85, 95",1x65,2020-03-07,12:15:00,90.7,NaN,2020-03-07 12:15:00,67.166667


In [136]:
# cardio specific processing
df_cardio['Average Pace Numeric'] = df_cardio['Average Pace'].apply(lambda x: (x.hour* 60 + x.minute) if pd.notnull(x) else False)
df_cardio.head(3)

,Rotation,Workout,Exercise,Week,Average Pace,Total Distance,Date,Time,Notes,Timestamp,Average Pace Numeric
workout_uid,,,,,,,,,,,
6_5.1,6.0,5.0,Run,1.0,05:58:00,6.25km,2020-02-25,20:10:00,NaN,2020-02-25 20:10:00,358
6_5.2,6.0,5.0,Run,2.0,05:40:00,6.30km,2020-03-04,18:22:00,NaN,2020-03-04 18:22:00,340
6_5.3,6.0,5.0,Run,3.0,06:01:00,6.31km,2020-03-09,19:47:00,NaN,2020-03-09 19:47:00,361


***
## Load Data <a class="anchor" id="load_data"></a>

In [137]:
%%time
# write to Tableau compliant format    

make_conversion(df_weightlifting, file_out_weightlifting)
clean_logs()

make_conversion(df_cardio, file_out_cardio)
clean_logs()

print("SYSTEM COMPLETE: {0}".format(
    datetime.datetime.strftime(datetime.datetime.now(), '%D %r')
))

(100, 15)


c:\python37\lib\site-packages\pandleau\pandleau.py:93: FutureWarning: A future version of pandas will default to `skipna=True`. To silence this warning, pass `skipna=True|False` explicitly.
  self._column_static_type = self._dataframe.apply(lambda x: pandleau.data_static_type(x), axis=0)
processing table: 0it [00:00, ?it/s]

Table 'Extract' does not exist in extract ./tableau/weightlifting_extract.hyper, creating.


processing table: 100it [00:00, 12456.36it/s]

PYTHON: ./tableau/weightlifting_extract.hyper conversion complete
PYTHON: 16/04/2020 16:32

PYTHON: Directory cleaned
PYTHON: 16/04/2020 16:32
(29, 11)



processing table: 29it [00:00, 14449.37it/s]

Table 'Extract' does not exist in extract ./tableau/cardio_extract.hyper, creating.
PYTHON: ./tableau/cardio_extract.hyper conversion complete
PYTHON: 16/04/2020 16:32

PYTHON: Directory cleaned
PYTHON: 16/04/2020 16:32
SYSTEM COMPLETE: 04/16/20 04:32:36 PM
Wall time: 477 ms
